Notebook to scrape country/language info from [Wikipedia page](https://en.wikipedia.org/wiki/List_of_countries_by_spoken_languages) add in ISO codes from [here](https://en.wikipedia.org/wiki/ISO_3166-1), join tables and write to DB

In [211]:
from bs4 import BeautifulSoup
import pandas as pd
from pandas.io import wb
import urllib2,re,sys
sys.path.append('/usr/local/lib/python2.7/dist-packages/')

In [680]:
import pymysql
from secrets import *

conn = pymysql.connect(host='localhost',user='root',passwd=PW, db='geo',charset='utf8')
cur = conn.cursor()

In [634]:
del conn
del cur

In [635]:
conn = pymysql.connect(host='localhost',user='root',passwd=PW, db='geo',charset='utf8')
cur = conn.cursor()

In [401]:
def cleanLanguage(l):
    if l=='Min/Min Nan':
        return u'Min'
    return re.sub(r'\(|\)','',l).strip()

### Grab Language by Country

In [590]:
wiki = "https://en.wikipedia.org/wiki/List_of_countries_by_spoken_languages"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)
 
headings=soup.findAll("h3")

In [591]:
tableData={}
tables = soup.findAll("table", { "class" : "wikitable"})
tables=tables[0:]

soFar=set()

for n,table in enumerate(tables): 
    # First few tables not languages
#    print n
    lang=table.findPrevious('h3').text.partition('[')[0]
    if lang in soFar:
        lang=table.findPrevious('h4').text.partition('[')[0]
    if lang in soFar:
        lang=table.findPrevious('h2').text.partition('[')[0]
    # Some language headings are h4 not h3
    
    if lang==u'Min/Min Nan':
        lang='Min'
    elif lang==u'Italic':
        lang=u'French'
        lang=table.findPrevious('h4').text.partition('[')[0]
        # hack
    
    soFar.add(lang)
    print '+++Adding',lang
    
    rows=[r for r in table.findAll('tr')[0].text.split('\n') if len(r)>0]
#    print rows
#   print ''
    
    data=[[] for r in rows]
    
    for n,row in enumerate(table.findAll('tr')[1:]):
                
        vals=[r for r in row.text.split('\n')]
#        print vals
        if len(vals[-1])==0:
            vals.pop(-1)
        if len(vals[0])==0 and n==0:
            vals.pop(0)
        if len(vals)>len(rows):
            vals.pop(0)
        
        if n==0:
            fillValue=vals[0]
            fillValue=cleanLanguage(fillValue)
#            rows[0]=fillValue
#            print fillValue,lang

#        print rows
#        print '=>'
#        print vals
#        print ''
        
        if u'Regions' in rows:
#            print 'Setting fillvalue to',lang,vals[0],fillValue
            vals[0]=fillValue
            
#            print vals
        
        if rows[0]=='Regions':
            vals[0]=lang
#            print vals
            
            
        if len(vals)==len(rows):
            for nn,val in enumerate(vals):
                data[nn].append(val)
        else:
            print 'Length error : ',vals

        try:
            if len(data[0][-1])==0:
                data[0][-1]=data[0][-2]
                # Fill forward
        except:
            print 'Cant fill forward',vals
#    print '--------------------------'
    tableData[lang]=(pd.DataFrame(data=dict(zip(rows,data))))
    print tableData[lang].shape
#    print ''

+++Adding Albanian
(9, 3)
+++Adding Armenian
(16, 3)
+++Adding Baltic
(7, 3)
+++Adding Celtic
(12, 3)
+++Adding Germanic
Length error :  [u'Low German', u'(Low Saxon)', u'\xa0Germany', u'Niederdeutsch, recognized as regional language']
(30, 3)
+++Adding Dutch
(15, 3)
+++Adding English
Length error :  [u'English', u'\xa0FS. Micronesia']
Length error :  [u'English', u'Dependent entities']
(81, 3)
+++Adding German
(21, 3)
+++Adding Greek
(14, 3)
+++Adding Iranian
(36, 3)
+++Adding Indo-Aryan
(21, 3)
+++Adding Domari
(29, 3)
+++Adding Romani
(52, 3)
+++Adding Other Indo-Aryan
(148, 3)
+++Adding French
(53, 3)
+++Adding Italian
(25, 3)
+++Adding Portuguese
(25, 3)
+++Adding Spanish
(37, 3)
+++Adding Other Italic Lang.
(29, 3)
+++Adding Slavic
Length error :  [u'Russian', u'', u'See also: List of countries where Russian is an official language', u'']
(109, 3)
+++Adding Mongolic languages
(4, 3)
+++Adding Turkic languages
(59, 3)
+++Adding Arabic
(43, 3)
+++Adding Other Semitic Lang.
(7, 3)
+

In [592]:
for k,v in tableData.items():
    print k
#    print v.columns
    v['name']=v['Country'].apply(unicode.strip)
    del v['Country']
    
    v.index=v.name
    
    if 'Regions' in v.columns:
        print '\t',k
        v['Language']=k
        v['type']=v['Regions']
        del v['Regions']
    else:
        v['type']=k
    

Bornean
Min
	Min
Cherokee
Celtic
East Fijian
Mandarin
	Mandarin
Dutch
	Dutch
Other Indo-Aryan
Dravidian languages
South Caucasian languages
	South Caucasian languages
Other Sino-Tibetan
Slavic
Uralic languages
Malayo-Sumbawan
Romani
	Romani
Tai–Kadai
French
	French
Mongolic languages
Domari
	Domari
Armenian
	Armenian
Polynesian
Apachean
Indo-Aryan
Philippines
Other Italic Lang.
Hmong–Mien
Germanic
Iranian
Language isolate
Albanian
Cantonese
	Cantonese
Yue Chinese
	Yue Chinese
Greek
English
	English
Baltic
Sunda–Sulawesi
Standard Chinese
Italian
	Italian
Portuguese
Japonic languages
Hakka
	Hakka
Turkic languages
German
	German
Micronesian
Other Semitic Lang.
Other Sinitic Lang.
Spanish
	Spanish
Arabic
	Arabic


In [593]:
tableData['Spanish']

,Status,name,Language,type
name,,,,
Spain,official,Spain,Spanish,Spanish
Andorra,majority,Andorra,Spanish,Spanish
France,minority,France,Spanish,Spanish
Germany,minority,Germany,Spanish,Spanish
Gibraltar,not official – majority,Gibraltar,Spanish,Spanish
Italy,minority,Italy,Spanish,Spanish
United Kingdom,minority,United Kingdom,Spanish,Spanish
Equatorial Guinea,co-official with Portuguese and French,Equatorial Guinea,Spanish,Spanish
Morocco,not official – minority,Morocco,Spanish,Spanish


In [433]:
iso.head()

,id,iso,iso2,iso3,name
name,,,,,
Afghanistan,004,ISO 3166-2:AF,AF,AFG,Afghanistan
Aland Islands !Åland Islands,248,ISO 3166-2:AX,AX,ALA,Aland Islands !Åland Islands
Albania,008,ISO 3166-2:AL,AL,ALB,Albania
Algeria,012,ISO 3166-2:DZ,DZ,DZA,Algeria
American Samoa,016,ISO 3166-2:AS,AS,ASM,American Samoa


### Grab ISO Codes/Country Names
[here](https://en.wikipedia.org/wiki/ISO_3166-1)

In [341]:
wiki = "https://en.wikipedia.org/wiki/ISO_3166-1"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

headings=soup.findAll("h3")


In [342]:
#tableData={}
tables = soup.findAll("table", { "class" : "wikitable"})
tables=tables[0:]

In [504]:
data=[]
rows=['name','iso2','iso3','id','iso']
data=[[] for k in rows]

for r in tables[0].findAll('tr'):
    for n,d in enumerate(r.findAll('td')):
#        print n,d.text
        data[n].append(d.text)
#    print ''

In [344]:
iso=pd.DataFrame(data=dict(zip(rows,data)))

In [345]:
iso.head()

,id,iso,iso2,iso3,name
0,004,ISO 3166-2:AF,AF,AFG,Afghanistan
1,248,ISO 3166-2:AX,AX,ALA,Aland Islands !Åland Islands
2,008,ISO 3166-2:AL,AL,ALB,Albania
3,012,ISO 3166-2:DZ,DZ,DZA,Algeria
4,016,ISO 3166-2:AS,AS,ASM,American Samoa


In [215]:
res=cur.execute('create table iso (name varchar(100) character set utf8,iso2 varchar(2),iso3 varchar(3))',)
# Create table of countries

In [218]:
cmd="INSERT INTO iso (name,iso2,iso3) VALUES (%s,%s,%s)"

for row in iso.iterrows():
    temp=(row[1][4],row[1][2],row[1][3])
    res=cur.execute(cmd,temp)
    conn.commit()

In [346]:
iso.head()

,id,iso,iso2,iso3,name
0,004,ISO 3166-2:AF,AF,AFG,Afghanistan
1,248,ISO 3166-2:AX,AX,ALA,Aland Islands !Åland Islands
2,008,ISO 3166-2:AL,AL,ALB,Albania
3,012,ISO 3166-2:DZ,DZ,DZA,Algeria
4,016,ISO 3166-2:AS,AS,ASM,American Samoa


In [347]:
iso.index=iso.name
# Useful for joining

In [348]:
iso.head()

,id,iso,iso2,iso3,name
name,,,,,
Afghanistan,004,ISO 3166-2:AF,AF,AFG,Afghanistan
Aland Islands !Åland Islands,248,ISO 3166-2:AX,AX,ALA,Aland Islands !Åland Islands
Albania,008,ISO 3166-2:AL,AL,ALB,Albania
Algeria,012,ISO 3166-2:DZ,DZ,DZA,Algeria
American Samoa,016,ISO 3166-2:AS,AS,ASM,American Samoa


### Join languages and countries

In [488]:
isoLangHash={}
langIsoHash={}
nameIsoHash={}

for row in iso.iterrows():
    isoLangHash[row[1][2]]=row[1][4]
    langIsoHash[row[1][4]]=row[1][2]
    nameIsoHash[row[1][4]]=row[1][2]

In [567]:
v.head()

,Status,name,Language,type,iso
name,,,,,
Islam,Liturgical language of Islam,Islam,Arabic,Arabic,None
Algeria,official,Algeria,Arabic,Arabic,DZ
Comoros,co-official with French,Comoros,Arabic,Arabic,KM
Chad,co-official with French,Chad,Arabic,Arabic,TD
Djibouti,co-official with French,Djibouti,Arabic,Arabic,DJ


In [608]:
for n,k in enumerate(tableData.keys()):
    t=tableData[k]
#    print '>>%s<<' % k
#    print '\t',t.columns
    t['iso']=t['name'].apply(lambda x:nameIsoHash.get(x))

In [595]:
tableData['Spanish']

,Status,name,Language,type,iso
name,,,,,
Spain,official,Spain,Spanish,Spanish,ES
Andorra,majority,Andorra,Spanish,Spanish,AD
France,minority,France,Spanish,Spanish,FR
Germany,minority,Germany,Spanish,Spanish,DE
Gibraltar,not official – majority,Gibraltar,Spanish,Spanish,GI
Italy,minority,Italy,Spanish,Spanish,IT
United Kingdom,minority,United Kingdom,Spanish,Spanish,GB
Equatorial Guinea,co-official with Portuguese and French,Equatorial Guinea,Spanish,Spanish,GQ
Morocco,not official – minority,Morocco,Spanish,Spanish,MA


In [653]:
combined=pd.concat(tableData.values())
# THis is final dataframe with all languages, iso codes and names

Manually add additions to iso dictionary

In [654]:
nameIsoHash['United States']='US'
nameIsoHash['United Kingdom']='GB'
nameIsoHash['Russia']='RU'
nameIsoHash['Iran']='IR'
nameIsoHash['Vietnam']='VN'
nameIsoHash['Burma']='MM'
nameIsoHash['Laos']='LA'
nameIsoHash['Brunei']='BN'
nameIsoHash['Syria']='SY'
nameIsoHash['FS. Micronesia']='FM'
nameIsoHash['Taiwan']='TW'
nameIsoHash['Macau']='MO'
nameIsoHash['Cape Verde']='CV'
nameIsoHash['Macedonia']='MK'
nameIsoHash['South Korea']='KR'
nameIsoHash['North Korea']='KP'
nameIsoHash['Moldova']='MD'
nameIsoHash['Hawaii']='US'
nameIsoHash['Tanzania']='TZ'
nameIsoHash['Palestine']='PS'
nameIsoHash['Venezuela']='VE'
nameIsoHash['Bolivia']='BO'

In [655]:
combined[pd.isnull(combined.iso)].name.value_counts()

Kosovo                                             5
Vatican City                                       3
Melilla                                            1
São Tomé and Príncipe                              1
Canary Islands                                     1
See also: Basque Country (historical territory)    1
British Virgin Islands                             1
Ceuta                                              1
Dem. Rep. Congo                                    1
Bolivia                                            1
Abkhazia                                           1
Sint Maarten                                       1
U.S. Virgin Islands                                1
Saint Barthélemy                                   1
Judaism                                            1
Falkland Islands                                   1
Scotland                                           1
American Tongan                                    1
Crimea  Ukraine                               

Put languages and countries in table

In [656]:
combined.name=combined.name.apply(unicode.lower)
combined.Language=combined.Language.apply(unicode)
combined.Language=combined.Language.apply(unicode.lower)

In [657]:
combined.head()

,Language,Status,iso,name,type
name,,,,,
Brunei,brunei malay,regional,BN,brunei,Bornean
Malaysia,brunei malay,minority,MY,malaysia,Bornean
Malaysia,coastal kadazan,minority,MY,malaysia,Bornean
Malaysia,iban,regional,MY,malaysia,Bornean
Brunei,iban,minority,BN,brunei,Bornean


Get language ISO codes

In [658]:
with open('iso_lang_codes.tsv','r') as inFile:
    lines=inFile.read().split('\n')

In [659]:
langToIso={}

for line in lines:
#    print line
    line=line.split('\t')
    langToIso[line[0].lower()]=line[1]

In [660]:
langToIso['english']='en'
langToIso['mandarin']='zh'
langToIso['cantonese']='zh'
langToIso['yue chinese']='zh'
# Put in manual language codes

In [661]:
combined['lang_iso']=combined['Language'].apply(lambda x:langToIso.get(x))

In [662]:
combined['Status']=combined['Status'].apply(unicode.lower)

In [663]:
combined[pd.isnull(combined.lang_iso)].Language.value_counts()

min                          52
romani                       52
hakka                        52
domari                       29
asia                         18
bhojpuri                     11
(miao)                       10
chittagonian                 10
africa                       10
rohingya                      9
south caucasian languages     9
europe                        9
telugu                        8
lomavren                      7
gilbertese                    6
slovene                       6
rusyn                         6
bosnian                       6
lahu[15]                      6
kyrgyz                        5
sylheti                       5
balochi                       4
karen[12]                     4
tausug                        4
south america                 4
brahui                        4
burmese[12]                   4
khmu[21]                      4
sami                          4
north america                 4
                             ..
manx    

### Make sense of the 'Status' column i.e. differentiate 'Official' from 'Minority'

In [687]:
def mapLangStatus(status):
    '''
    Takes string description of language staus
    and returns integer representation of prevalence
    e.g. "Official" => 1
    "minority" => 2
    All others => 3
    '''
    
    if re.search(r'official|de facto',status) and not re.search(r'minority',status):
        return 1
    elif re.search(r'minority|regional|significant|majority',status):
        return 2
    else:
        return 3

In [688]:
combined['lang_level']=combined['Status'].apply(lambda x:mapLangStatus(x))

### Write data to language table in DB

In [690]:
res=cur.execute('create table language (language varchar(100) character set utf8, country_name varchar(100) character set utf8, iso2 varchar(2),status varchar(50),lang_iso varchar(2),level smallint)')

In [691]:
cmd="INSERT INTO language (language,country_name,iso2,status,lang_iso,level) VALUES (%s,%s,%s,%s,%s,%s)"

        
for row in combined.iterrows():
    print row[0],row[1].values
    temp=(row[1]['Language'],row[1]['name'],row[1]['iso'],row[1]['Status'],row[1]['lang_iso'],row[1]['lang_level'])
    res=cur.execute(cmd,temp)
    conn.commit()


Brunei [u'brunei malay' u'regional' 'BN' u'brunei' u'Bornean' None 2]
Malaysia [u'brunei malay' u'minority' u'MY' u'malaysia' u'Bornean' None 2]
Malaysia [u'coastal kadazan' u'minority' u'MY' u'malaysia' u'Bornean' None 2]
Malaysia [u'iban' u'regional' u'MY' u'malaysia' u'Bornean' None 2]
Brunei [u'iban' u'minority' 'BN' u'brunei' u'Bornean' None 2]
Indonesia [u'iban' u'minority' u'ID' u'indonesia' u'Bornean' None 2]
Comoros [u'malagasy' u'minority' u'KM' u'comoros' u'Bornean' 'MG' 2]
Madagascar [u'malagasy' u'official' u'MG' u'madagascar' u'Bornean' 'MG' 1]
Mayotte [u'malagasy' u'minority' u'YT' u'mayotte' u'Bornean' 'MG' 2]
Angola [u'min' u'minority' u'AO' u'angola' 'Min' None 2]
Cape Verde [u'min' u'minority' 'CV' u'cape verde' 'Min' None 2]
Mauritius [u'min' u'minority' u'MU' u'mauritius' 'Min' None 2]
South Africa [u'min' u'minority' u'ZA' u'south africa' 'Min' None 2]
Brunei [u'min' u'minority' 'BN' u'brunei' 'Min' None 2]
Cambodia [u'min' u'minority' u'KH' u'cambodia' 'Min' None

In [666]:
combined[combined.name=='angola']

,Language,Status,iso,name,type,lang_iso
name,,,,,,
Angola,min,minority,AO,angola,Min,None
Angola,mandarin,minority,AO,angola,Mandarin,zh
Angola,yue chinese,minority,AO,angola,Yue Chinese,zh
Angola,africa,official,AO,angola,Portuguese,None
Angola,hakka,minority,AO,angola,Hakka,None


In [678]:
combined.to_csv('country_language.csv',encoding='utf-8')

In [676]:
combined[combined['lang_level']==2]['Status'].value_counts()

minority                                                                                                                           723
significant minority                                                                                                                63
regional                                                                                                                            45
minority, from immigration                                                                                                           8
minority[3]                                                                                                                          5
recognized regional lang.                                                                                                            4
significant language                                                                                                                 3
minority, endangered dravidian language                